In [ ]:
import SimpleITK as sitk
def readMRIVolume(mri_volume_path):
    reader = sitk.ImageSeriesReader()
    dicom_files = reader.GetGDCMSeriesFileNames(mri_volume_path)
    #'dicom_files' are the individual slices of an MRI volume
    reader.SetFileNames(dicom_files)
    retrieved_mri_volume = reader.Execute()
    return retrieved_mri_volume


In [ ]:

  # Resample images to uniform voxel spacing
import numpy as np  
def resample_image(input_volume, out_spacing=[1, 1, 1]):
  
    original_spacing = input_volume.GetSpacing()
    original_size = input_volume.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(input_volume.GetDirection())
    resample.SetOutputOrigin(input_volume.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(input_volume.GetPixelIDValue())

    return resample.Execute(input_volume)


In [ ]:

def resize_image(input_volume):
    # Resize images to fixed spatial resolution in pixels
    num_axial_slices = int(input_volume.GetSize()[-1])
    output_size = [320, 320, num_axial_slices]
    scale = np.divide(input_volume.GetSize(), output_size)
    spacing = np.multiply(input_image.GetSpacing(), scale)
    transform = sitk.AffineTransform(3)
    resized_volume = sitk.Resample(input_volume, output_size, transform, sitk.sitkLinear, input_volume.GetOrigin(),
                                  spacing, 
    input_volume.GetDirection())
    return resized_volume

In [ ]:

def extract_slices(image_volume):
    image_array = sitk.GetArrayFromImage(image_volume)
    image_slices_array = image_array[int(np.shape(image_array)[0]):int(np.shape(image_array)[0]),:,:]
    return image_slices_array


In [ ]:

image_slice = sitk.GetImageFromArray(image_slices_array)
sitk.WriteImage(image_slice, 'image_slice.png')

In [ ]:

##############DATA_SPLIT & TRANSFORM CODE: START#####################################
from sklearn.model_selection import train_test_split
import torch.utils
import numpy as np
from torch.utils.data import Dataset, DataLoader, Subset

import numpy as np
train_transforms = transforms.Compose([
                            transforms.RandomHorizontalFlip(p=0.1),
                            transforms.Resize(224),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=mean, std=std)])

val_transforms = transforms.Compose([
                                    transforms.Resize(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=mean, std=std)])

# adapted from ptrblck post
# custom dataset class in order to apply different transforms to different sets of data
class LoadCustomDataSet(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        if self.transform:
            x = self.transform(img_dataset[index][0])
        else:
            x = img_dataset[index][0]
        y = img_dataset[index][1]
        return x, y

    def __len__(self):
        return len(img_dataset)

batch_size = 32

# path to the directory which contains data folders belonging to various classes
data_dir =  ''
img_dataset = datasets.ImageFolder(data_dir)
targets = img_dataset.targets
# num_classes would be useful later while fine-tuning the pre-trained network
num_classes = len(img_dataset.classes)

# employing stratified train_test_split to make sure train and validation set have balanced data.
train_id, valid_id= train_test_split(
np.arange(len(targets)),
test_size=0.2,
shuffle=True,
stratify=targets)

train_set = LoadCustomDataSet(img_dataset, train_transforms)
val_set = LoadCustomDataSet(img_dataset, val_transforms)

train_data = Subset(train_set, indices=train_id)
val_data = Subset(val_set, indices=valid_id)

train_data_len = (len(train_id))
val_data_len = (len(valid_id))

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)

val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
##############DATA_SPLIT & TRANSFORM CODE: END#####################################

In [ ]:

from __future__ import print_function, division
import torch
from torchvision import datasets, transforms
import os
# path of the directory where the data is located
data_dir = ''

data_transforms = {
# folder_name is the name of the root directory: assuming the data has not been split yet!
    'folder_name': transforms.Compose([
        transforms.ToTensor()
    ]),
}

image_dataset = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['folder_name']}

dataloaders = {x: torch.utils.data.DataLoader(image_dataset[x], batch_size=20,
                                             shuffle=False, num_workers=4)
              for x in ['folder_name']}


mean = 0.
std = 0.
total_samples = 0.
# batch_samples are the samples being used at one point in time
# data.size(1) is the number of channels in the data
for input_images,_ in dataloaders['folder_name']:
    batch_samples = input_images.size(0)
    # Here, view is used to reshape the data and flatten the last two dimensions (320X320) size of images used in earlier code 
    input_images = input_images.view(batch_samples, input_images.size(1), -1)
    mean += input_images.mean(2).sum(0)
    std += input_images.std(2).sum(0)
    total_samples += batch_samples
    
# calculation of final mean and standard deviation values 
mean /= total_samples
std /= total_samples

In [ ]:

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import time

# use cuda if GPU support is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
    since = time.time()
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()

                running_loss = 0.0
                running_corrects = 0.0

                i = 0
                # Iterate over data.
                for inputs,labels in (train_loader):
                    # inputs, labels = data
                    # print(data)
                    # print(labels)
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                    # loss and accuracy
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += (torch.sum(preds == labels.data))
                    # print("running corrects ",running_corrects)

                scheduler.step()

                train_loss = running_loss/train_data_len
                # print(running_corrects)
                train_acc = running_corrects/train_data_len


                print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(
                    train_loss, train_acc))

            else:
                # choose eval() mode for validation phase
                val_acc, val_loss = eval_model(model, val_loader)
                print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(
                    val_loss, val_acc))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # print('Best test Acc: {:4f}'.format(best_acc))


def eval_model(model, val_loader):
    running_loss = 0.0
    running_corrects = 0.0
    model.eval()
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += float(torch.sum(preds == labels.data))
    val_loss = running_loss / val_data_len
    val_acc = running_corrects / val_data_len

    return val_acc, val_loss


# finetuning the model
num_epochs = 25
transfer_learn_network = models.resnet34(pretrained=True)
num_ftrs = transfer_learn_network.fc.in_features


transfer_learn_network.fc = nn.Linear(num_ftrs,len(classes))
transfer_learn_network = transfer_learn_network.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(transfer_learn_network.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
policy = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# model training and evaluation
train_model(transfer_learn_network, criterion, optimizer, policy,
            num_epochs=num_epochs)


In [ ]:

# finetuning the model
model_ft = models.densenet121(pretrained=True)
num_ftrs = model_ft.classifier.in_features

model_ft.classifier = nn.Linear(num_ftrs, len(classes))
model_ft = model_ft.to(device)

In [ ]:

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# training and evaluating the model
def train_model(model, criterion, optimizer, scheduler, num_epochs=2):
    since = time.time()
    conf_matrix_data = []
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                label_list = torch.zeros(train_data_len)
                prediction_list = torch.zeros(train_data_len)

                running_loss = 0.0
                running_corrects = 0.0

                i = 0
                # Iterate over data.
                for inputs,labels in (train_loader):
                   
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':

                            loss.backward()
                            optimizer.step()
                            label_list[i:i + (len(labels))] = labels
                            prediction_list[i:i + (len(labels))] = preds
                            i += len(labels)
                            package_label = zip(label_list, prediction_list)
                            conf_matrix_data.append(package_label)
                  
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += (torch.sum(preds == labels.data))
 

                scheduler.step()

                train_loss = running_loss/train_data_len

                train_acc = running_corrects/train_data_len


                print('Train Loss: {:.4f} Train Acc: {:.4f}'.format(
                    train_loss, train_acc))

            else:
                # choose eval() mode for validation phase
                val_acc, val_loss = eval_model(model, val_loader)
                print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(
                    val_loss, val_acc))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    return conf_matrix_data
    # print('Best test Acc: {:4f}'.format(best_acc))


def eval_model(model, val_loader):
    running_loss = 0.0
    running_corrects = 0.0
    model.eval()
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += float(torch.sum(preds == labels.data))
    val_loss = running_loss / val_data_len
    val_acc = running_corrects / val_data_len

    return val_acc, val_loss


# finetuning the model
# set the number of epochs
num_epochs = 25
transfer_learn_network = models.resnet34(pretrained=True)
num_ftrs = transfer_learn_network.fc.in_features


transfer_learn_network.fc = nn.Linear(num_ftrs,len(classes))
transfer_learn_network = transfer_learn_network.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.SGD(transfer_learn_network.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
policy = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# model training and evaluation
conf_matrix_data = train_model(transfer_learn_network, criterion, optimizer, policy,
            num_epochs=num_epochs)

############################CONFUSION MATRIX CODE: START####################################################################
confusion_matrix = torch.zeros(2,2)
for i in range(len(conf_matrix_data)):
    confusion_matrix = torch.zeros(2, 2)
    for targ,pred in conf_matrix_data[i]:
        predict_temp = pred.type(torch.LongTensor)
        target_temp = targ.type(torch.LongTensor)
        confusion_matrix[target_temp, predict_temp] += 1

display_conf_matrix = []
TP = confusion_matrix.diag()
for class_num in range(len(classes)):
    id = torch.ones(2).byte()
    id[class_num] = 0
    # all non-class samples classified as non-class
    TN = confusion_matrix[
        id.nonzero()[:,
        None], id.nonzero()].sum()
    FP = confusion_matrix[id, class_num].sum()
    FN = confusion_matrix[class_num, id].sum()

    Specificity = TN/(TN+FP)
    Sensitivity = TP[class_num]/(TP[class_num] + FN)

    display_conf_matrix.append([[int(TP[class_num]), int(FN)], [int(FP), int(TN)]])
    print('Specificity of {} Class , {}'.format(
        class_num,Specificity))
    print('Sensitivity of {} Class , {}'.format(
        class_num, Sensitivity))
    print('Class {}\nTP {}, TN {}, FP {}, FN {}'.format(
        class_num, TP[class_num], TN, FP, FN))


import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

# display class-wise confusion matrix
for count in range(len(classes)):
    df_cm = pd.DataFrame(display_conf_matrix[count], index = ['Class '+str(count),'non-Class'],
                  columns = ['Class '+str(count),'non-Class'])
    title_str = "Confusion matrix for class " + str(count)

    plt.figure(figsize = (10,7))

    sn.heatmap(df_cm, annot=True)
    plt.suptitle(title_str)
    plt.ylabel('Actual Class')
    plt.xlabel('Predicted Class')
    plt.show()
############################CONFUSION MATRIX CODE: END####################################################################

